In [ ]:
#you can either use tabula or pdfplumber to extract tabula data from a pdf
#let's install tabula to our environment
pip install tabula-py

In [ ]:
#import libraries
import tabula
import pandas as pd

In [ ]:
#define the path
pdf_path = '/content/MPESA_Statement_2021-08-17_to_2023-08-17_2547xxxxxx053.pdf'

In [ ]:
#convert the pdf to a list of data frames
data = tabula.read_pdf(pdf_path, pages = 'all')

In [ ]:
#this is the dataframe i want to use so I create this as the reference
dfs = data[2]

In [ ]:
#dfs.to_csv('pesa.csv', index = True)

In [ ]:
#just to see and confirm the data is okay
dfs.head(30)

In [ ]:
#create an empty list that will carry the matching dataframes in relation to the reference columns
same_dfs = []


In [ ]:
#iterate through the list
#check for column length similarity between the dfs
#append
for df in data:
  if len(df.columns) == len(dfs.columns):
    same_dfs.append(df)

In [ ]:
#combine the dfs
combined_df = pd.concat(same_dfs, ignore_index = True)

In [ ]:
#tjust to confirm
combined_df.count()

Receipt No.           2254
Completion Time       2254
Details               2254
Transaction Status    2254
Paid In                264
Withdrawn             1990
Balance               2254
Unnamed: 0               0
dtype: int64

In [ ]:

combined_df.tail(50)

In [ ]:
#Details column is quite lomg and unique for similar transactions
#functions to categorise
def cat_details(details):
  if "Pay Bill Online to" in details or "Pay Bill to" in details:
    return "Paybill"
  elif "Business Payment from" in details:
    return "Business Payment from"
  elif "Funds received from" in details:
    return "Funds received from"
  elif "KCB M-PESA Withdraw" in details:
    return "KCB M-PESA Withdraw"
  elif "Offnet C2B Transfer" in details:
    return "Offnet C2B Transfer"
  elif "M-shwari Withdraw" in details:
    return "M-Shwari Withdraw"
  elif "Customer Transfer" in details:
    return "Customer Transfer"
  elif "Withdrawal Charge" in details:
    return "Withdrawal Charge"
  elif "Deposit of Funds" in details:
    return "Deposit of Funds"
  elif "Merchant Payment" in details:
    return "Merchant Payment"
  elif "Pay Bill Charge" in details:
    return "Paybill Charge"
  elif "Customer Payment to Small" in details:
    return "Customer Payment to"
  elif "Airtime Purchase" in details:
    return "Airtime Purchase"
  elif "Offnet C2B Transaction" in details:
    return "Offnet C2B Reversal"
  elif "Customer Withdrawal At Agent" in details:
    return "Withdrawal At Agent"
  elif "Send Money Reversal" in details:
    return "Send Money Reversal"
  elif "Pay Utility Reversal" in details:
    return "Pay Utility Reversal"
  elif "FSI LSA Statement" in details:
    return "FSI LSA Statement"
  elif "FI Organization Savings" in details:
    return "FI Organization Savings"
  elif "Salary Payment" in details:
    return "Salary Payment"
  else:
    return details

combined_df['Category'] = combined_df['Details'].apply(cat_details)


In [ ]:
combined_df['Category'].nunique()

26

In [ ]:
combined_df['Category'].unique()

array(['Airtime Purchase', 'Offnet C2B Transfer', 'Funds received from',
       'M-Shwari Withdraw', 'M-Shwari Deposit', 'Deposit of Funds',
       'Paybill Charge', 'Paybill', 'Customer Transfer',
       'Withdrawal Charge', 'Withdrawal At Agent',
       'Business Payment from', 'Offnet C2B Reversal',
       'Pay Utility Reversal', 'Merchant Payment', 'FSI LSA Statement',
       'M-Shwari Lock Deposit', 'Customer Payment to',
       'KCB M-PESA Withdraw', 'Send Money Reversal',
       'FI Organization Savings', 'Salary Payment', 'KCB M-PESA Deposit',
       'M-Shwari Lock Deposit from\rSavings',
       'Merchant Customer Payment\rfrom 7198278 - Teddy\rWilliam     Otieno   11',
       'M-Shwari Lock Activate and Save\rfrom Savings'], dtype=object)

In [ ]:

null_rows = combined_df.isnull().all(axis=1)
null_count = null_rows.sum()
null_count

0

In [ ]:
#we dont need the details column
combined_df.drop('Details', axis = 1, inplace = True)

In [ ]:
#also drop this unused column
combined_df.drop('Unnamed: 0', axis = 1, inplace = True)

In [ ]:
combined_df.columns

Index(['Receipt No.', 'Completion Time', 'Transaction Status', 'Paid In',
       'Withdrawn', 'Balance', 'Category'],
      dtype='object')

In [ ]:
#fill nulls in paid column wit 0
combined_df['Paid In'].fillna(0 , inplace = True)

In [ ]:
#same to withdrawn
combined_df['Withdrawn'].fillna(0, inplace = True)

In [ ]:
#convert to csv and download
combined_df.to_csv('mpesa.csv', index = False, header = True)